S0.data download

In [ ]:
# read meta file
import pandas as pd
import wget
import os
import tarfile
import shutil

tcga_src = pd.read_csv('./TCGA/TCGA_data_table.tsv', sep='\t')

In [ ]:
# create folders for data download
import os
parent_dir = '/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/TCGA/'
for cohort in tcga_src['Cohort']:
    path = os.path.join(parent_dir, cohort)
    os.mkdir(path)
    print("Directory '%s' created" % cohort)  

In [ ]:
# data download
parent_dir = '/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/TCGA/'
for id in tcga_src['Cohort']:
    path = os.path.join(parent_dir, id)
    url = f'http://gdac.broadinstitute.org/runs/stddata__2016_01_28/data/{id}/20160128/gdac.broadinstitute.org_{id}.Merge_rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__exon_quantification__data.Level_3.2016012800.0.0.tar.gz'
    out_name = id + '.tar.gz'
    file_name = wget.download(url, out=os.path.join(path, out_name))
    print(file_name)

In [ ]:
# data organization

TCGA_path = '/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/TCGA/'
for dir in os.listdir(TCGA_path):
    current_dir = os.path.join(TCGA_path, dir)
    print(current_dir)
    file_path = os.path.join(current_dir, dir+'.tar.gz')
    if os.path.exists(file_path):
        print('Unzip ' + dir + '.tar.gz')
        with tarfile.open(file_path, 'r') as tar:
            # Extract all the contents
            tar.extractall(path=current_dir)
        # get sub folder path
        sub_path = ''
        for d in os.listdir(current_dir):
            if 'gdac' in d:
                sub_path = os.path.join(current_dir, d)
        print(sub_path)
        print(os.listdir(sub_path)[1])
        data_path = os.path.join(sub_path, os.listdir(sub_path)[1])
        print(data_path)
        new_data_path = os.path.join(current_dir, dir + '_exon.tsv')
        print(new_data_path)
        shutil.move(data_path, new_data_path)
        for d in os.listdir(current_dir):
            if 'gdac' in d:
                sub_path = os.path.join(current_dir, d)
                print('remove dir:')
                print(sub_path)
                shutil.rmtree(sub_path)

S1. format expr matrix

In [1]:
import pandas as pd
import subprocess

# function to exact Exon expression matrix from raw data table
def get_expr_matrix(file_path, data_types):
    col_idx = []
    raw_expr = pd.read_csv(file_path, sep='\t', skiprows=[1], index_col="Hybridization REF")
    if data_types == 'raw_reads':
        col_idx = list(range(0,len(raw_expr.columns),3))
    elif data_types == 'median_length_normalize':
        col_idx = list(range(1,len(raw_expr.columns),3))
    elif data_types == 'RPKM':
        col_idx = list(range(2,len(raw_expr.columns),3))
    else:
        raise Exception("Sorry, wrong data types")
    sub_expr = raw_expr.iloc[:,col_idx]
    # strip column labels
    if data_types == 'median_length_normalize':
        sub_expr.columns = sub_expr.columns.str.rstrip('.1')
    elif data_types == 'RPKM':
        sub_expr.columns = sub_expr.columns.str.rstrip('.2')
    return sub_expr


def get_exon_bed(expr_matrix, cancer_type, outpath):
    out_file = os.path.join(outpath, cancer_type+'_exon.bed')
    expr_bed = open(out_file, 'w')
    for i in expr_matrix.index:
        chr = i.split(':')[0]
        if '_rCRS' in chr:
            chr = chr.rstrip('_rCRS')
        coords = i.split(':')[1].split('-')
        start = int(coords[0])
        end = int(coords[1])
        strand = i.split(':')[2]
        # print(i)
        expr_bed.write(f"{chr}\t{start}\t{end}\t{i}\t{cancer_type}\t{strand}\n")
    expr_bed.close()
    return out_file

def get_SEP_expr(exon_intersection_path, expr_matrix, SEP_IDs, outpath):
    if os.path.exists(exon_intersection_path):
        exon_overlap = pd.read_csv(exon_intersection_path, sep='\t', header=None)
        exon_expr = expr_matrix.loc[exon_overlap[9].to_list()]
        exon_expr['ID'] = exon_overlap[3].to_list()
        SEP_expr = exon_expr.groupby('ID').sum()
        SEP_with_exons = SEP_expr.index.to_list()
        for sep in SEP_IDs:
            if sep not in SEP_with_exons:
                SEP_expr.loc[sep] = [0] * len(SEP_expr.columns)
        SEP_expr_st = SEP_expr.sort_index()
        SEP_expr_st.to_csv(outpath, sep='\t')
        return SEP_expr_st

def intersect_bed_files(cancer_bed, sep_bed, intersect_bed):
    cmd = f'bedtools intersect -a {sep_bed} -b {cancer_bed} -wa -wb -s'
    result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    with open(intersect_bed, 'w') as file: 
        file.write(result.stdout)
    file.close()
    
            
    
    
    




In [3]:
import os
sep_bed_path = '/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/bed/candis_hg19.bed'
bed_path='/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/bed'
expr_out = './SEP_exon_expr' 
TCGA_DB = './TCGA'
cancer_types = os.listdir(TCGA_DB)
sep_df = pd.read_csv(sep_bed_path, sep='\t', header = None)
sep_ls = sep_df.loc[:,3].to_list()
print(sep_ls)

['IP_661448', 'IP_080226', 'IP_688853', 'IP_722311', 'II_710341', 'IP_635221', 'IP_755087', 'IP_190092', 'IP_288216', 'IP_600934', 'IP_081904', 'IP_665954', 'IP_167118', 'SEP119', 'ASDURF', 'DWORF', 'NBDY']


In [4]:
# print(cancer_types)
for cancer in cancer_types[21:]:
    raw_expr_path = os.path.join(TCGA_DB, cancer, cancer+'_exon.tsv')
    if os.path.exists(raw_expr_path):
        print(cancer)
        expr_matrix = get_expr_matrix(raw_expr_path, 'raw_reads')
        expr_matrix_bed = get_exon_bed(expr_matrix, cancer, bed_path)
        print(expr_matrix_bed)
        intersect_bed_path = os.path.join(bed_path, 'intersection', cancer+'_ov.bed')
        intersect_bed_files(expr_matrix_bed, sep_bed_path, intersect_bed_path)
        if os.path.getsize(intersect_bed_path) != 0:
            sep_expr_out_path = os.path.join(expr_out, cancer+'_exon_expr.tsv')
            sep_expr = get_SEP_expr(intersect_bed_path, expr_matrix, sep_ls, sep_expr_out_path)
        else:
            print('The no exon overlap with sep!')          
    else:
        print(cancer+' no exist!')

MESO
/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/bed/MESO_exon.bed
OV
/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/bed/OV_exon.bed
PAAD
/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/bed/PAAD_exon.bed
PCPG
/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/bed/PCPG_exon.bed
PRAD
/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/bed/PRAD_exon.bed
READ
/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/bed/READ_exon.bed
SARC
/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/bed/SARC_exon.bed
SKCM
/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/bed/SKCM_exon.bed
STAD
/home/likecheng/projects/Cooperation/30_SEP_fun

S2.get bed files

In [41]:
candidates = ['IP_661448','IP_080226','IP_688853','IP_722311','II_710341','IP_635221','IP_755087','IP_190092','IP_288216','IP_600934','IP_081904', 'IP_665954','IP_167118', 'II_740871']


In [ ]:

import pandas as pd

openprot_bed = pd.read_pickle('/home/likecheng/projects/orf_db/openprot/openprot_bed.pkl')
# print(openprot_bed.head())
openprot_bed.index = openprot_bed.name
cand_bed = openprot_bed.loc[candidates, ]

In [ ]:
ates = ['IP_661448','IP_080226','IP_688853','IP_722311','II_710341','IP_635221','IP_755087','IP_190092','IP_288216','IP_600934','IP_081904', 'IP_665954','IP_167118', 'II_740871']
import pandas as pd

openprot_bed = pd.read_pickle('/home/likecheng/projects/orf_db/openprot/openprot_bed.pkl')
# print(openprot_bed.head())
openprot_bed.index = openprot_bed.name
cand_bed = openprot_bed.loc[candidates, ]

In [31]:
cand_bed.to_csv('./bed/candis.bed', index=None, sep='\t', header=None)

s3. get expression matrix

In [22]:
import pandas as pd
exon_overlap = pd.read_csv('./bed/intersected_exon.tsv', sep='\t', header=None)

In [59]:
exon_expr = expr_RPKM.loc[exon_overlap[9].to_list()]

In [60]:
exon_expr['ID'] = exon_overlap[3].to_list()

In [61]:
exon_expr.groupby('ID').sum()


,TCGA-CS-4938-01B-11R-1896-07,TCGA-CS-4941-01A-01R-1470-07,TCGA-CS-4942-01A-01R-1470-07,TCGA-CS-4943-01A-01R-1470-07,TCGA-CS-4944-01A-01R-1470-07,TCGA-CS-5390-01A-02R-1470-07,TCGA-CS-5393-01A-01R-1470-07,TCGA-CS-5394-01A-01R-1470-07,TCGA-CS-5395-01A-01R-1470-07,TCGA-CS-5396-01A-02R-1470-07,...,TCGA-VW-A8FI-01A-11R-A36H-07,TCGA-W9-A837-01A-11R-A36H-07,TCGA-WH-A86K-01A-11R-A36H-07,TCGA-WY-A858-01A-11R-A36H-07,TCGA-WY-A859-01A-12R-A36H-07,TCGA-WY-A85A-01A-21R-A36H-07,TCGA-WY-A85B-01A-11R-A36H-07,TCGA-WY-A85C-01A-11R-A36H-07,TCGA-WY-A85D-01A-11R-A36H-07,TCGA-WY-A85E-01A-11R-A36H-07
ID,,,,,,,,,,,,,,,,,,,,,
ASDURF,66.382811,58.820675,77.814551,100.534607,61.172027,90.483210,77.600247,79.608651,66.965451,97.193230,...,38.250577,84.715831,51.376427,33.536131,50.362312,55.921282,78.900098,71.540366,53.661958,71.755422
DWORF,0.068094,0.599161,13.164843,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.082910,0.000000,0.000000,0.000000
II_710341,0.669808,2.495306,0.941282,0.659173,0.706428,1.164024,1.269304,1.728226,1.127051,3.248178,...,0.393912,1.856985,1.247746,1.468459,2.237631,1.248733,1.131239,1.031836,1.190476,2.218923
IP_080226,0.302231,0.754280,0.286228,1.792313,0.297144,0.607132,0.530730,0.297543,0.345793,0.343190,...,2.163524,0.822863,1.394571,1.911386,2.451781,1.184209,1.602942,3.492483,0.963748,1.955480
IP_167118,22.636689,13.350401,40.684173,159.171157,17.648819,46.455598,93.819406,69.871161,13.459547,49.286817,...,9.379191,37.279271,43.678700,38.731827,29.876454,52.247789,62.449130,40.405415,30.760849,67.849463
IP_190092,0.000000,0.160809,0.000000,0.010986,0.024307,0.005644,0.000000,0.021946,0.011786,0.338940,...,0.017507,0.011790,0.205977,0.000000,0.000000,0.008122,0.042093,0.286865,0.572977,0.334706
IP_288216,24.050175,2.979592,48.499817,19.902110,9.424237,16.047814,25.908849,44.087978,12.836946,5.629711,...,64.758666,33.878812,102.645883,44.205617,14.606704,17.142937,42.132711,15.428285,76.215509,19.462876
IP_688853,10.011903,7.829443,8.411415,27.443144,8.378314,19.577256,15.336106,16.467198,15.335448,22.268017,...,5.300143,16.535306,8.802531,6.224705,15.594858,13.141727,16.731737,19.524856,9.794881,15.108420
NBDY,23.716080,30.418186,30.163437,43.344924,27.623535,38.276860,30.525030,35.042115,12.930767,42.798165,...,24.110008,28.810672,37.258218,26.626545,36.768139,32.838336,34.446949,26.882022,27.652935,36.019517


merge all cancer types

In [41]:
import os
sep_expr_out_path = '/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/SEP_exon_expr'
print(os.listdir(sep_expr_out_path))

['ACC_exon_expr.tsv', 'BLCA_exon_expr.tsv', 'BRCA_exon_expr.tsv', 'CESC_exon_expr.tsv', 'CHOL_exon_expr.tsv', 'COADREAD_exon_expr.tsv', 'COAD_exon_expr.tsv', 'DLBC_exon_expr.tsv', 'ESCA_exon_expr.tsv', 'GBMLGG_exon_expr.tsv', 'GBM_exon_expr.tsv', 'HNSC_exon_expr.tsv', 'KICH_exon_expr.tsv', 'KIPAN_exon_expr.tsv', 'KIRC_exon_expr.tsv', 'KIRP_exon_expr.tsv', 'LAML_exon_expr.tsv', 'LGG_exon_expr.tsv', 'LIHC_exon_expr.tsv', 'LUAD_exon_expr.tsv', 'LUSC_exon_expr.tsv', 'MESO_exon_expr.tsv', 'OV_exon_expr.tsv', 'PAAD_exon_expr.tsv', 'PCPG_exon_expr.tsv', 'PRAD_exon_expr.tsv', 'READ_exon_expr.tsv', 'SARC_exon_expr.tsv', 'SKCM_exon_expr.tsv', 'STAD_exon_expr.tsv', 'STES_exon_expr.tsv', 'TGCT_exon_expr.tsv', 'THCA_exon_expr.tsv', 'THYM_exon_expr.tsv', 'UCEC_exon_expr.tsv', 'UCS_exon_expr.tsv', 'UVM_exon_expr.tsv']


In [42]:
cancer_exprs = pd.DataFrame()
for cancer_f in os.listdir(sep_expr_out_path):
    cancer = cancer_f.split('_')[0]
    print(cancer)
    file_path = os.path.join(sep_expr_out_path, cancer_f)
    df = pd.read_csv(file_path, sep='\t', index_col='ID')
    df = df.T
    df['cancer_types'] = cancer
    print(len(df))
    cancer_exprs = pd.concat([cancer_exprs, df])
    

ACC
79
BLCA
427
BRCA
1212
CESC
309
CHOL
45
COADREAD
433
COAD
328
DLBC
48
ESCA
196
GBMLGG
701
GBM
171
HNSC
566
KICH
91
KIPAN
1020
KIRC
606
KIRP
323
LAML
173
LGG
530
LIHC
423
LUAD
576
LUSC
552
MESO
87
OV
307
PAAD
183
PCPG
187
PRAD
550
READ
105
SARC
265
SKCM
473
STAD
450
STES
646
TGCT
156
THCA
568
THYM
122
UCEC
201
UCS
57
UVM
80


In [ ]:
cancer_exprs

In [44]:
import numpy as np
for sep in cancer_exprs.columns[:-1]:
    cancer_exprs[sep] = np.log2(cancer_exprs[sep].astype(float) + 1)

In [45]:

cancer_exprs['patient_ID'] = cancer_exprs.index.str[:12]

def extract_sample_type(barcode):
    if int(barcode[13:15])  < 10:
        return 'Tumor'
    elif int(barcode[13:15]) >= 10 :
        return 'Normal'
    else:
        return 'Unknown'

# Apply the function to the index and create a new column
cancer_exprs['sample_type'] = cancer_exprs.index.map(extract_sample_type)

In [54]:
sep_ls = cancer_exprs.columns[:-3]

S4.Visualization

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

out_path = '/home/likecheng/projects/Cooperation/30_SEP_function_minning_20241104/s1_get_expression_matrixs/image_output'


for sep in sep_ls:
	# Create the violin plot
	plt.figure(figsize=(15, 6))
	sns.violinplot(x='cancer_types', y=sep, hue='sample_type', data=cancer_exprs, split=True,inner="quart")

	# Add titles and labels
	plt.title(f'Violin plot of {sep} expression levels by Cancer Types')
	plt.xlabel('Cancer Types')
	plt.ylabel(f'log2(Rawreads+1)')
	plt.legend(title='Sample Types')
	plt.xticks(rotation=90)
	# Save the plot as a PNG file
	save_path = os.path.join(out_path, sep+'.png')
	plt.savefig(save_path, dpi=300, bbox_inches='tight')  # Save with 300 dpi


